In [1]:
from pyspark.sql.types import *
#from pyspark.sql.types import sqlContext

In [2]:
import pandas as pd
import time

In [3]:
# Optimize the convertion of pandas and spark dataframes
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 1000)

In [4]:
redirect_sdf = sqlContext.read.parquet("redirect.parquet")
pageid_sdf = sqlContext.read.parquet("pageid.parquet")

In [5]:
redirect_sdf.printSchema()
pageid_sdf.printSchema()

root
 |-- pageId: long (nullable = true)
 |-- pageTitle: string (nullable = true)
 |-- __index_level_0__: long (nullable = true)

root
 |-- pageId: long (nullable = true)
 |-- pageTitle: string (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [6]:
redirect_sdf.registerTempTable('redirect')
pageid_sdf.registerTempTable('pageid')

In [7]:
#temp_df =  sqlContext.sql("select * from redirect")

In [8]:
#type(temp_df)

In [9]:
#temp_df.take(5)

In [10]:
query = """
select
pageId, pageTitle from redirect limit 5
"""
sqlContext.sql(query).toPandas()

,pageId,pageTitle
0,10,Computer_accessibility
1,13,History_of_Afghanistan
2,14,Geography_of_Afghanistan
3,15,Demographics_of_Afghanistan
4,18,Communications_in_Afghanistan


In [11]:
sqlContext.sql('SELECT * FROM pageid LIMIT 10').show()

+------+--------------------+-----------------+
|pageId|           pageTitle|__index_level_0__|
+------+--------------------+-----------------+
|    10| AccessibleComputing|                0|
|    12|           Anarchism|                1|
|    13|  AfghanistanHistory|                2|
|    14|AfghanistanGeography|                3|
|    15|   AfghanistanPeople|                4|
|    18|AfghanistanCommun...|                5|
|    19|AfghanistanTransp...|                6|
|    20| AfghanistanMilitary|                7|
|    21|AfghanistanTransn...|                8|
|    23| AssistiveTechnology|                9|
+------+--------------------+-----------------+



In [12]:
redirect_sdf = redirect_sdf.withColumnRenamed('pageTitle','fix_target')

In [13]:
redirect_sdf = redirect_sdf.drop('__index_level_0__')
pageid_sdf = pageid_sdf.drop('__index_level_0__')

In [14]:
redirect_sdf = redirect_sdf.join(pageid_sdf,'pageId',how='left')
redirect_sdf = redirect_sdf.withColumnRenamed('pageTitle','initial_target')
redirect_sdf = redirect_sdf.drop('pageId')

In [15]:
#redirect = redirect_sdf.persist()

In [16]:
redirect_sdf.take(5)

[Row(fix_target='Demographics_of_Albania', initial_target='AlbaniaPeople'),
 Row(fix_target='Brachycephaly', initial_target='Brachycephalic'),
 Row(fix_target='Executive_(government)', initial_target='Executive_power'),
 Row(fix_target='Endangered_species', initial_target='Endangered_Species'),
 Row(fix_target='Evolution', initial_target='Evolved')]

In [17]:
redirect_sdf.registerTempTable('redirect')

In [18]:
query = """
select * from redirect limit 5
"""
sqlContext.sql(query).toPandas()

,fix_target,initial_target
0,Demographics_of_Albania,AlbaniaPeople
1,Brachycephaly,Brachycephalic
2,Executive_(government),Executive_power
3,Endangered_species,Endangered_Species
4,Evolution,Evolved


In [19]:
#query = """
#select * from redirect limit 10
#"""
#sqlContext.sql(query).toPandas()

In [20]:
pagelinks_sdf = sqlContext.read.parquet("/home/benjamin/wikipedia/Wikipedia/pagelinks_parsed0.parquet")

In [21]:
pagelinks_sdf = pagelinks_sdf.withColumnRenamed('pageTitle','target')
pagelinks_sdf = pagelinks_sdf.drop('__index_level_0__')

In [22]:
# turn source ids to titles
pagelinks_sdf = pagelinks_sdf.join(pageid_sdf, on='pageId', how='inner')
pagelinks_sdf = pagelinks_sdf.withColumnRenamed('pageTitle','source')
pagelinks_sdf = pagelinks_sdf.drop('pageId')

In [23]:
#pagelinks_sdf.persist()
#pagelinks_sdf.take(5)

In [24]:
#pagelinks_sdf = pagelinks_sdf.update(redirect_sdf)
pagelinks_sdf = pagelinks_sdf.join(redirect_sdf,pagelinks_sdf.target == redirect_sdf.initial_target,how='left')
pagelinks_sdf =  pagelinks_sdf.drop('initial_target')
pagelinks_sdf.persist()

DataFrame[target: string, source: string, fix_target: string]

In [25]:
pagelinks_sdf.take(5)

[Row(target='!HERO', source='Skillet_discography', fix_target='!Hero'),
 Row(target='(100678)_1997_XV9', source='Beijing_Schmidt_CCD_Asteroid_Program', fix_target='List_of_minor_planets:_100001–101000'),
 Row(target='(119878)_2001_CY224', source='Resonant_trans-Neptunian_object', fix_target=None),
 Row(target='(12009)_1996_UE', source='List_of_Mars-crossing_minor_planets', fix_target=None),
 Row(target='(14415)_1991_RQ7', source='Hiroshi_Kaneda', fix_target='List_of_minor_planets:_14001–15000')]

In [30]:
pagelinks_sdf.select('source',(pagelinks_sdf['source'] != pagelinks_sdf['target']).alias('change')).take(5)

[Row(source='Skillet_discography', change=True),
 Row(source='Beijing_Schmidt_CCD_Asteroid_Program', change=True),
 Row(source='Hiroshi_Kaneda', change=True),
 Row(source='List_of_Jupiter_trojans_(Greek_camp)_(1–100000)', change=True),
 Row(source='List_of_Jupiter_trojans_(Greek_camp)_(1–100000)', change=True)]

In [26]:
from pyspark.sql.functions import udf

In [27]:
def update_col(l,r):
    if r:
        return r
    else:
        return l
update_udf = udf(update_col)

In [28]:
update_udf = udf(update_col)

In [33]:
pagelinks_sdf.select('target','fix_target', update_udf('target','fix_target').alias('fix_target2')).take(10)

AnalysisException: "cannot resolve '`target`' given input columns: [source, fix_target];;\n'Project ['target, fix_target#136, update_col('target, fix_target#136) AS fix_target2#176]\n+- AnalysisBarrier\n      +- Project [source#63, update_col(target#54, fix_target#28) AS fix_target#136]\n         +- Project [target#54, source#63, fix_target#28]\n            +- Join LeftOuter, (target#54 = initial_target#39)\n               :- Project [target#54, source#63]\n               :  +- Project [pageId#48L, target#54, pageTitle#7 AS source#63]\n               :     +- Project [pageId#48L, target#54, pageTitle#7]\n               :        +- Join Inner, (pageId#48L = pageId#6L)\n               :           :- Project [pageId#48L, target#54]\n               :           :  +- Project [pageId#48L, pageTitle#49 AS target#54, __index_level_0__#50L]\n               :           :     +- Relation[pageId#48L,pageTitle#49,__index_level_0__#50L] parquet\n               :           +- Project [pageId#6L, page

In [30]:
pagelinks_sdf = pagelinks_sdf.select('source', update_udf('target','fix_target').alias('fix_target'))

In [31]:
pagelinks_sdf.write.parquet('pagelinks_corrected0spark.parquet')

In [32]:
pagelinks_sdf.take(5)

[Row(source='Skillet_discography', fix_target='!Hero'),
 Row(source='Beijing_Schmidt_CCD_Asteroid_Program', fix_target='List_of_minor_planets:_100001–101000'),
 Row(source='Resonant_trans-Neptunian_object', fix_target='(119878)_2001_CY224'),
 Row(source='List_of_Mars-crossing_minor_planets', fix_target='(12009)_1996_UE'),
 Row(source='Hiroshi_Kaneda', fix_target='List_of_minor_planets:_14001–15000')]